In [4]:
import os

def chunk_file(filepath, lines_per_chunk=10000):
    with open(filepath, 'r') as infile:
        chunk_count = 0
        for _ in range(1, 10):
            chunk_count += 1
            output_filename = f"chuncks/chunck_{chunk_count}.txt"
            
            with open(output_filename, 'w') as outfile:
                for i in range(lines_per_chunk):
                    line = infile.readline()
                    if not line:
                        return
                    outfile.write(line)

chunk_file('bookcorpus100mb.txt')

In [7]:
import chromadb
import uuid
import os
import time
import statistics

client = chromadb.PersistentClient(path="./chroma_data")
collection = client.get_or_create_collection(name="bookcorpus")
times = []

MAX_BATCH_SIZE = 1000

def insert_chromadb(chunck_id):
    with open(f"chuncks/chunck_{chunck_id}.txt", "r", encoding="utf-8") as f:
        bookcorpus_lines: list[str] = f.read().splitlines()
    start_time = time.time()
    for j in range(0, len(bookcorpus_lines), MAX_BATCH_SIZE):
        batch_documents = bookcorpus_lines[j:j + MAX_BATCH_SIZE]
        collection.add(
            ids=[str(uuid.uuid4()) for _ in batch_documents],
            documents=batch_documents
        )
    end_time = time.time()
    times.append(end_time - start_time)

def performance():
    min_time = min(times)
    max_time = max(times)
    avg_time = sum(times) / len(times)
    std_dev = statistics.stdev(times) if len(times) > 1 else 0
    print(f"Minimum time: {min_time:.6f} seconds")
    print(f"Maximum time: {max_time:.6f} seconds")
    print(f"Average time: {avg_time:.6f} seconds")
    print(f"Standard deviation: {std_dev:.6f} seconds")

if __name__ == '__main__':
    for i in range (1,10):
        insert_chromadb(i)
    performance()


/home2/users/alumnes/1279420/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 11.6MiB/s]


Minimum time: 268.333851 seconds
Maximum time: 394.667032 seconds
Average time: 300.466737 seconds
Standard deviation: 49.805606 seconds


In [25]:
import chromadb
import uuid
import os
import time
import statistics

client = chromadb.PersistentClient(path="./chroma_data")
collection = client.get_or_create_collection(name="bookcorpus")
times = []

MAX_BATCH_SIZE = 1000

def query_chroma(query_text1, query_text2):
    start = time.time()
    results = collection.query(
        query_texts= [
            query_text1,
            query_text2
        ],
        n_results=3 # Retrieve the top 2 most similar results
    )
    end = time.time()
    times.append(end - start)
    return results

if __name__ == '__main__':
    texts = collection.peek()["documents"] # returns the first 10
    embs = collection.peek()["embeddings"]
    for i in range(0,10):
        print(texts[i])
        qtext1 = f"top-2 most similar sentence using euclidean distance as metric for {texts[i]}"
        qtext2 = f"top-2 most similar sentence using Manhattan distance as metric for {texts[i]}"

        results = query_chroma(qtext1, qtext2)
        for j, query_results in enumerate(results["documents"]):
            print(f"\nQuery {j}")
            print("\n".join(query_results))
        print()
        
    performance()

usually , he would be tearing around the living room , playing with his toys .

Query 0
usually , he would be tearing around the living room , playing with his toys .
he could n't remember the last time he 'd spent an entire day indoors , and within an hour he was pacing in circles around the living room .
walking into her space , he could n't help but notice how little furniture she had .

Query 1
usually , he would be tearing around the living room , playing with his toys .
he could n't remember the last time he 'd spent an entire day indoors , and within an hour he was pacing in circles around the living room .
having made the decision , she calculated the distance .

but just one look at a minion sent him practically catatonic .

Query 0
but just one look at a minion sent him practically catatonic .
`` distant ?
distance simply was n't 'done ' in this line of business .

Query 1
`` distant ?
distance simply was n't 'done ' in this line of business .
having made the decision , she c

In [3]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_data")
print("Number of documents that can be inserted at once: ",client.get_max_batch_size())

Number of documents that can be inserted at once:  5461
